In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from dataloader.rosen import RosenData
from experiment_setup import get_model, set_random, build_estimator
from analysis.metrics import uq_accuracy, uq_ndcg, uq_nll
from uncertainty_estimator.masks import BasicMask, LHSMask, MirrorMask, DecorrelationMask

config = {
    'random_seed': 43,
    'n_dim': 10,
    'nn_runs': 100,
    'runs': 2,
    'model_runs': 5,
    'data_size': 2000,
    'data_split': [0.4, 0.6, 0, 0],
    'verbose': True,
    'use_cache': True,
    'layers': [10, 128, 64, 32, 1],
    'epochs': 30_000,
    'acc_percentile': 0.1,
    'patience': 5,
    'target_loss': 750,
    'dropout_rate': 0.3
}


In [ ]:
rosen = RosenData(
    config['n_dim'], config['data_size'], config['data_split'],
    use_cache=config['use_cache'])

x_pool, y_pool = rosen.dataset('pool')
x_train, y_train = rosen.dataset('train')
x_val, y_val = rosen.dataset('train')

# set_random(config['random_seed'])


In [ ]:
model_paths = [f"model/data/rosen_visual_{i}.ckpt" for i in range(config['model_runs'])]
if not config['use_cache']:
    for i in range(config['model_runs']):
        model = get_model(
            config['layers'], model_paths[i],
            (x_train, y_train), (x_val, y_val), epochs=config['epochs'],
            retrain=True, verbose=False, patience=config['patience'])
        print(f"Loss {model.val_loss}/{config['target_loss']}")


In [ ]:
config['model_runs'] = 5
#     'nn_runs': 100,
#     'runs': 2,
masks = {
    'vanilla': None,
    'basic_mask': BasicMask(),
    'lhs': LHSMask(config['nn_runs']),
    'lhs_shuffled': LHSMask(config['nn_runs'], shuffle=True),
    'mirror_random': MirrorMask(),
    'decorrelating': DecorrelationMask(),
    'decorrelating_scaled': DecorrelationMask(scaling=True, dry_run=False)
}

mask_results = []

for model_run in range(config['model_runs']):
    print(f"===MODEL RUN {model_run+1}====")
    model = get_model(
        config['layers'], model_paths[model_run],
        (x_train, y_train), (x_val, y_val), epochs=config['epochs'])   
    predictions = model(x_val).cpu().numpy()
    errors = np.abs(predictions - y_val)

    for name, mask in masks.items():
        estimator = build_estimator(
            'mcdue_masked', model, nn_runs=config['nn_runs'], dropout_mask=mask,
            dropout_rate=config['dropout_rate'])

        for run in range(config['runs']):
            estimations = estimator.estimate(x_val, x_train, y_train)
            acc = uq_accuracy(estimations, errors, config['acc_percentile'])
            ndcg = uq_ndcg(errors, estimations)
            nll = uq_nll(errors, estimations)
            mask_results.append([acc, ndcg, nll, name])

            if hasattr(mask, 'reset'):
                mask.reset()


In [ ]:
mask_df = pd.DataFrame(mask_results, columns = ['acc', 'ndcg', 'nll', 'mask'])

def boxplot(df, x_label, y_label, bottom=0, top=1):
    plt.figure()
    plt.xticks(rotation=30)
    if bottom is not None:
        plt.ylim(bottom, top)
    sns.boxplot(data=mask_df, x=x_label, y=y_label)
    
boxplot(mask_df, 'mask', 'acc')
boxplot(mask_df, 'mask', 'ndcg')
boxplot(mask_df, 'mask', 'nll', None)